In [ ]:
###############################################################################################################
##################################### Diagnostics #############################################################
###############################################################################################################

In [ ]:
# Findings!
# The sample ids have hierarchy:
# Case UUID: becedbfd-b2aa-4dde-b7f4-29e6f59ec32c in data bank
# Sample UUID: f6ee8148-9fb7-404e-b951-d1af8d06974f not in data bank
# Belongs to sample TCGA-E2-A1IK-01A
# Sample UUID: 8577ac01-1274-4bd5-ab04-380eaa78d95b in data bank
# Belongs to aliquot TCGA-E2-A1IK-01A-11D-A17G-09
# TCGA-E2-A1IK-01A
#     Portions
#     TCGA-E2-A1IK-01A-11
#         Analytes
#         TCGA-E2-A1IK-01A-11D
#             Aliquots
#             TCGA-E2-A1IK-01A-11D-A141-01
#             TCGA-E2-A1IK-01A-11D-A17G-09
#             TCGA-E2-A1IK-01A-11D-A140-02
#             TCGA-E2-A1IK-01A-11D-A142-09
#             TCGA-E2-A1IK-01A-11D-A145-05

# Solution: find sample UUID of aliquot sample UUID -> R package TCGAutils

# Sample and case id counts
# Case hits
#  Single hits: 9775 
# Multiple hits: 125 
# No hit: 208 
# Total: 10108 

# Sample Hits:
#  Single hits: 10003 
# Multiple hits: 0 
# No hit: 243 
# Total: 10108

# COSMIC is in TCGA
# 118 - COSMIC	Overlapping COSMIC variants


In [1]:
###############################################################################################################
############################################### Setup #########################################################
###############################################################################################################

import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient()
db = client.progenetix
bs = db.biosamples

# Read in dataframe from intermediate and mapping file
df = pd.read_csv('../temp/mappingfile.tsv', sep = '\t') 
cases = list(set(df['case_id']))
samples = list(set(df['sample_id']))

In [2]:
###############################################################################################################
##################################### Sample and case counts ##################################################
###############################################################################################################

hits_c = []

for ids in cases:
    n_hits_c = bs.count_documents({"external_references.id": {'$regex': ids} ,"biosample_status.id":"EFO:0009656"})
    hits_c.append(n_hits_c)
    
c_single_hit = 0
c_multiple_hits = 0
c_no_hit = 0

for i in hits_c:
    if i == 1:
        c_single_hit += 1
    if i == 0:
        c_no_hit += 1
    if i != 0 and i != 1:
        c_multiple_hits += 1

print("Case hits",  "\nSingle hits:", c_single_hit, "\nMultiple hits:", c_multiple_hits,
      "\nNo hit:", c_no_hit, "\nTotal:", len(hits_c), "\n")

hits_s = []

for ids in samples:
    n_hits_s = bs.count_documents({"external_references.id": {'$regex': ids} ,"biosample_status.id":"EFO:0009656"})
    hits_s.append(n_hits_s)

s_single_hit = 0
s_multiple_hits = 0
s_no_hit = 0

for i in hits_s:
    if i == 1:
        s_single_hit += 1
    if i == 0:
        s_no_hit += 1
    if i != 0 and i != 1:
        s_multiple_hits += 1

print("Sample Hits:", "\nSingle hits:", s_single_hit, "\nMultiple hits:", s_multiple_hits,
      "\nNo hit:", s_no_hit, "\nTotal:", len(hits_c))

# Case hits
#  Single hits: 9775 
# Multiple hits: 125 
# No hit: 208 
# Total: 10108 

# Sample Hits:
#  Single hits: 10003 
# Multiple hits: 0 
# No hit: 243 
# Total: 10108

Case hits 
Single hits: 9775 
Multiple hits: 125 
No hit: 208 
Total: 10108 

Sample Hits: 
Single hits: 10003 
Multiple hits: 0 
No hit: 243 
Total: 10108


In [18]:
###############################################################################################################
##################################### HGVSc Problem ###########################################################
###############################################################################################################

# Filter rows with variant_type == 'SNP'
dd_filtered = df.loc[df['variant_type'] == 'SNP', ['hgvsc', 'reference_bases', 'alternate_bases']].copy()

# Drop rows with missing values in HGVSC
dd_filtered.dropna(subset=['hgvsc'], inplace=True)

# Extract the last character from HGVSC and assign it to a new column 'HGVSC_ref_base'
dd_filtered['HGVSC_ref_base'] = dd_filtered['hgvsc'].str.split('>').str[0].str[-1]
dd_filtered['HGVSC_alt_base'] = dd_filtered['hgvsc'].str.split('>').str[1]


# Reset the index if necessary
dd_filtered.reset_index(drop=True, inplace=True)

compare = dd_filtered[['hgvsc', 'HGVSC_ref_base', 'reference_bases', 'HGVSC_alt_base', 'alternate_bases']]
issues = compare[compare['reference_bases'] != compare['HGVSC_ref_base']]

compare

,hgvsc,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
0,c.1760C>G,C,G,G,C
1,c.103G>C,G,G,C,C
2,c.658A>T,A,A,T,T
3,c.1380C>A,C,C,A,A
4,c.3484C>A,C,C,A,A
...,...,...,...,...,...
2424420,c.813G>A,G,G,A,A
2424421,c.1022T>G,T,A,G,C
2424422,c.165T>A,T,T,A,A
2424423,c.56G>T,G,G,T,T


In [6]:
print(
    'Different alt base =\t', sum(issues['HGVSC_ref_base'] != issues['alternate_bases']),
    '\nSame alt base =\t\t', sum(issues['HGVSC_ref_base'] == issues['alternate_bases']),
    '\nTotal =\t\t\t', len(issues)
)

Different alt base =	 1076223 
Same alt base =		 138916 
Total =			 1215139


In [8]:
print('Number of matching alt bases:',
      sum(issues['HGVSC_alt_base'] == issues['alternate_bases']) # alt base is never alt base
     )

print('Number of alt bases that are not the ref base:',  
      sum(issues['HGVSC_alt_base'] != issues['reference_bases']) # same numbers as above
     )

Number of matching alt bases: 0
Number of alt bases that are not the ref base: 1076223


In [19]:
issues

,hgvsc,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
0,c.1760C>G,C,G,G,C
7,c.1160C>G,C,G,G,C
8,c.1339A>G,A,T,G,C
11,c.227A>T,A,T,T,A
14,c.2113C>A,C,G,A,T
...,...,...,...,...,...
2424409,c.292T>C,T,A,C,G
2424410,c.1148G>C,G,C,C,G
2424416,c.254C>A,C,G,A,T
2424418,c.869G>A,G,C,A,T


In [24]:
same_alt = issues[issues['HGVSC_ref_base'] == issues['alternate_bases']]
# sum(same_alt['HGVSC_alt_base'] == same_alt['reference_bases']), len(same_alt) 
# switched hgvs alt base with seq ref base

(138916, 138916)

In [26]:
diff_alt = issues[issues['HGVSC_ref_base'] != issues['alternate_bases']]
diff_alt

,hgvsc,HGVSC_ref_base,reference_bases,HGVSC_alt_base,alternate_bases
8,c.1339A>G,A,T,G,C
14,c.2113C>A,C,G,A,T
16,c.1393C>A,C,G,A,T
18,c.954T>C,T,A,C,G
19,c.2112C>T,C,G,T,A
...,...,...,...,...,...
2424408,c.2785G>A,G,C,A,T
2424409,c.292T>C,T,A,C,G
2424416,c.254C>A,C,G,A,T
2424418,c.869G>A,G,C,A,T


In [39]:
diff_alt.loc[diff_alt['HGVSC_ref_base'] == 'T', 'comp_ref'] = 'A'
diff_alt.loc[diff_alt['HGVSC_ref_base'] == 'A', 'comp_ref'] = 'T'
diff_alt.loc[diff_alt['HGVSC_ref_base'] == 'G', 'comp_ref'] = 'C'
diff_alt.loc[diff_alt['HGVSC_ref_base'] == 'C', 'comp_ref'] = 'G'
sum(diff_alt['reference_bases'] == diff_alt['comp_ref']), len(diff_alt) 
# all the  hgvs ref bases are complementary to the seq ref base
diff_alt.loc[diff_alt['HGVSC_alt_base'] == 'T', 'comp_alt'] = 'A'
diff_alt.loc[diff_alt['HGVSC_alt_base'] == 'A', 'comp_alt'] = 'T'
diff_alt.loc[diff_alt['HGVSC_alt_base'] == 'G', 'comp_alt'] = 'C'
diff_alt.loc[diff_alt['HGVSC_alt_base'] == 'C', 'comp_alt'] = 'G'
sum(diff_alt['alternate_bases'] == diff_alt['comp_alt']), len(diff_alt) 
# all the hgvs alt bases are complementary to seq alt base

(1076223, 1076223)

In [40]:
len(dd_filtered)

2424425

In [ ]:
# Alt base is sometimes hgvsc ref base, but never hgvsc alt base
# Ref base is never hgvsc ref base, but sometimes hgvsc alt base
# Where HGVS ref base is SEQ alt base: switched alt and ref
# Where HGVS ref base is not SEQ alt base: complementary bases

# From 2424425 SNP entries are 1215139 problematic.
# Of the 1215139 problems:
# - 1076223 instances have complementary hgvs and seq bases
# - 138916 instances have switched the ref and alt base

# HGVSc is just unreliable..
# Is HGVSc less reliable than seq data? -> probably